# For Windows
1. Please download the latest SasView v5 package from
   https://github.com/SasView/sasview/tags
2. Install SasView
3. Adjust the *sasviewProgramPath* variable accordingly for the path where SasView was installed.

## In case of error
<pre style="background: #fdd">
    CalledProcessError: Command '['gcc', ...
</pre>
Please start the previously installed SasView at least once and create a plot with the desired model function. This will build the selected model from C-sources into a python library in the directory <br />
`C:\Users\{user}\.sasmodels\compiled_models\sas32_{model}.so`

If it still fails, it might be caused by OpenCL being installed while the *pyopencl* package can not be imported in jupyter notebook/lab. Try to disable OpenCL entirely by setting `os.environ["SAS_OPENCL"] = "none"` early in the code and check that the desired SasView model was build already (see above).

# McSAS example

In [ ]:
# these need to be loaded at the beginning to avoid errors related to relative imports (ImportWarning in h5py)
# might be related to the change of import style for Python 3.5+. Tested on Python 3.7 at 20200417
import sys, os, pandas, numpy, scipy 
from pathlib import Path
# these packages are failing to import in McHat if they are not loaded here:
import h5py
from scipy.special import j0
import scipy.optimize


%matplotlib inline
import matplotlib.pyplot as plt
from mcsas3 import McData1D

import warnings
warnings.filterwarnings('error')

In [ ]:
def plotResults(measData, mcAna):
    rect = [0.1, 0.1, 0.8, 0.8]
    a1 = plt.axes(rect)
    for i in range(len(histRanges)):
        mode = mcAna.averageMode(i)
        mcAna.debugPlot(i, alpha=0.5,
            label="hist. {}={:.1f}".format(
                histRanges.iloc[i].parameter.title(),
                mode['mean'].valMean))
    def setAxes(ax, vert, horiz):
        for ax, loc in (ax.xaxis, vert), (ax.yaxis, horiz):
            ax.set_label_position(loc)
            getattr(ax, 'tick_'+loc)()
    setAxes(a1, 'top', 'right')
    plt.xlabel("R (nm)")
    plt.ylabel("Frequency")

    a2=plt.axes(rect, frameon=False)
    setAxes(a2, 'bottom', 'left')
    barWidth = numpy.diff(measData["Q"][0])
    barWidth = numpy.array((barWidth[0],) + tuple(barWidth))*.5
    plt.errorbar(measData["Q"][0], mcAna.modelIAvg.modelIMean,
                 yerr=mcAna.modelIAvg.modelIStd,
                 elinewidth=1, label="MC fit", zorder=2)
    plt.plot(measData["Q"][0], measData["I"],
             label="Input Data", linewidth=4, zorder=1)
    plt.yscale("log")
    plt.xscale("log")
    plt.xlabel("Q (1/nm)")
    plt.ylabel("I (1/m)")

    combined = [x for a in (a2, a1)
                  for x in zip(*a.get_legend_handles_labels())]
    a2.legend(*list(zip(*combined)), loc=4)

In [ ]:
# load required modules

homedir = os.path.expanduser("~")
# disable OpenCL for multiprocessing on CPU
os.environ["SAS_OPENCL"] = "none"
# set location where the SasView/sasmodels are installed
# sasviewPath = os.path.join(homedir, "AppData", "Local", "SasView")
sasviewPath = os.path.join(homedir, "Code", "sasmodels") # BRP-specific
if sasviewPath not in sys.path:
    sys.path.append(sasviewPath)
from mcsas3 import McHat

# Spheres example

In [ ]:
mds = McData1D.McData1D(filename = Path('src', 'testdata', 'quickstartdemo1.csv'), nbins = 100, csvargs={'sep' : ';', "header": None, "names": ["Q", "I", "ISigma"]})

In [ ]:
fh, (ah) = plt.subplots()
mds.rawData.plot('Q', 'I', ax = ah)
mds.binnedData.plot('Q', 'I', yerr = 'IError', ax = ah, logx = True, logy = True)

In [ ]:
# run the Monte Carlo method
mh = McHat.McHat(
    modelName="sphere", nContrib=300, modelDType="default",
    fitParameterLimits={"radius": (1, 314)},
    staticParameters={"background": 0, "scaling": 0.1e6},
    maxIter=1e5, convCrit=1, nRep=40, nCores=0, seed=None)
md = mds.measData.copy()


In [ ]:
mh.run(md, "resultssphere.h5")

In [ ]:
# histogram the determined size contributions
from mcsas3.mcmodelhistogrammer import McModelHistogrammer
from mcsas3.mcanalysis import McAnalysis
histRanges = pandas.DataFrame([dict(
                parameter="radius", nBin=50, binScale="log",
                presetRangeMin=1, presetRangeMax=314,
                binWeighting="vol", autoRange=True),
                dict(
                parameter="radius", nBin=50, binScale="linear",
                presetRangeMin=10, presetRangeMax=100,
                binWeighting="vol", autoRange=False),
                ])
mcres = McAnalysis("resultssphere.h5", md, histRanges, store=True)

In [ ]:
# plotting the intensity match and histogram
plt.figure(figsize=(7,5))
plotResults(md, mcres)

# Gaussian chain example:

In [ ]:
# # remove any prior results file:
# resPath = Path("test_resultsgaussianchain.h5")
# if resPath.is_file:
#     resPath.unlink()

# mdg = McData1D.McData1D(filename=r"S2870 BSA THF 1 1 d.pdh", dataRange = [0.1, 4], nbins = 50)
# # load required modules
# homedir = os.path.expanduser("~")
# # disable OpenCL for multiprocessing on CPU
# os.environ["SAS_OPENCL"] = "none"
# # set location where the SasView/sasmodels are installed
# # sasviewPath = os.path.join(homedir, "AppData", "Local", "SasView")
# sasviewPath = os.path.join(homedir, "Code", "sasmodels")  # BRP-specific
# if sasviewPath not in sys.path:
#     sys.path.append(sasviewPath)
# from mcsas3 import McHat

# # run the Monte Carlo method
# mh = McHat.McHat(
#     modelName="mono_gauss_coil",
#     nContrib=300,
#     modelDType="default",
#     fitParameterLimits={"rg": (1, 20)},
#     staticParameters={"background": 0, "i_zero": 0.00319},
#     maxIter=1e5,
#     convCrit=2,
#     nRep=5,
#     nCores=0,
#     seed=None,
# )
# # test step seems to be broken? Maybe same issue with multicore processing with sasview
# mh.run(mdg.measData, "test_resultsgaussianchain.h5")

# # histogram the determined size contributions
# from mcsas3.mcmodelhistogrammer import McModelHistogrammer
# from mcsas3.mcanalysis import McAnalysis
# histRanges = pandas.DataFrame([dict(
#                 parameter="rg", nBin=25, binScale="linear",
#                 presetRangeMin=0.1, presetRangeMax=30,
#                 binWeighting="vol", autoRange=False),])
# mcresg = McAnalysis("test_resultsgaussianchain.h5", mdg.measData, histRanges, store=True)
# fh, (ah) = plt.subplots()
# mdg.rawData.plot('Q', 'I', ax = ah)
# mdg.binnedData.plot('Q', 'I', yerr = 'IError', ax = ah, logx = True, logy = True)
# # plotting the intensity match and histogram
# plt.figure(figsize=(7,5))
# plotResults(mdg.measData, mcresg)